# Train

In [53]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os, glob, cv2
import numpy as np
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam

In [4]:
train_path = '../dataset/images/'
classes = ["100", "118", "217"]

In [25]:
def load_data_cnn(train_path):
    images = []
    labels = []
    classes = [lists for lists in os.walk(train_path)][0][1]    
    for cls in classes:
        file_path = os.path.join(train_path, cls, "*g")
        for file in glob.glob(file_path):
            image = cv2.imread(file)
            image = image[288-250:288-35, 55:390]
            image = cv2.resize(image, (int(215/5), int(215/5)), cv2.INTER_LINEAR)
            label = np.zeros(len(classes))
            label[classes.index(cls)] = 1.0
            images.append(image)
            labels.append(label) 
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

dataset = load_data_cnn(train_path)

In [74]:
X_train = []
X_train.append(dataset[0][:80])
X_train.append(dataset[0][100:180])
X_train.append(dataset[0][200:280])
X_train = [j for i in X_train for j in i]
X_train = np.array(X_train)

y_train = []
y_train.append(dataset[1][:80])
y_train.append(dataset[1][100:180])
y_train.append(dataset[1][200:280])
y_train = [j for i in y_train for j in i]
y_train = np.array(y_train)

X_test = []
X_test.append(dataset[0][80:100])
X_test.append(dataset[0][180:200])
X_test.append(dataset[0][280:300])
X_test = [j for i in X_test for j in i]
X_test = np.array(X_test)

y_test = []
y_test.append(dataset[1][80:100])
y_test.append(dataset[1][180:200])
y_test.append(dataset[1][280:300])
y_test = [j for i in y_test for j in i]
y_test = np.array(y_test)

y_test_label = []
for i in range(60):
    if i >= 40:
        y_test_label.append(2)
    elif i >= 20:
        y_test_label.append(1)
    else:
        y_test_label.append(0)
        

In [77]:
X_train[0].shape

(43, 43, 3)

In [29]:
input_shape = (int(215/5), int(215/5), 3)

In [46]:
cnn_model = Sequential()

cnn_model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=input_shape))
cnn_model.add(Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
cnn_model.add(Conv2D(32, (3, 3), activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(3, activation='softmax'))

In [47]:
cnn_model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
classifier = cnn_model.fit(X_train, y_train, validation_split=0.1, epochs=60, batch_size=9)

Train on 216 samples, validate on 24 samples
Epoch 1/60
216/216 [==============================] - 2s 7ms/step - loss: 1.0487 - acc: 0.4954 - val_loss: 1.0718 - val_acc: 0.2917
Epoch 2/60
216/216 [==============================] - 1s 7ms/step - loss: 1.0558 - acc: 0.4815 - val_loss: 0.9697 - val_acc: 0.8333
Epoch 3/60
216/216 [==============================] - 1s 7ms/step - loss: 0.9538 - acc: 0.5741 - val_loss: 0.7202 - val_acc: 1.0000
Epoch 4/60
216/216 [==============================] - 2s 7ms/step - loss: 0.8760 - acc: 0.5926 - val_loss: 0.5007 - val_acc: 1.0000
Epoch 5/60
216/216 [==============================] - 2s 7ms/step - loss: 0.6912 - acc: 0.7407 - val_loss: 0.3928 - val_acc: 1.0000
Epoch 6/60
216/216 [==============================] - 2s 7ms/step - loss: 0.6389 - acc: 0.7546 - val_loss: 0.2641 - val_acc: 1.0000
Epoch 7/60
216/216 [==============================] - 2s 7ms/step - loss: 0.5591 - acc: 0.7963 - val_loss: 0.2061 - val_acc: 1.0000
Epoch 8/60
216/216 [===========

In [70]:
loss, accuracy = cnn_model.evaluate(X_test, y_test)
print("Loss = "+str(loss))
print("Accuracy = "+str(accuracy))

60/60 [==============================] - 0s 2ms/step
Loss = 0.004240412854899963
Accuracy = 1.0


In [51]:
predictions = cnn_model.predict_classes(X_test, batch_size=5)

In [71]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [78]:
cn = confusion_matrix(y_test_label, predictions)

In [80]:
print ("Confusion Matrix")
cn

Confusion Matrix


array([[20,  0,  0],
       [ 0, 20,  0],
       [ 0,  0, 20]])

In [81]:
cnn_model.save("../models/cnn_3labels_1.h5")

# Test

In [82]:
from keras.models import load_model

In [83]:
trained_model = load_model("../models/cnn_3labels_1.h5")

In [84]:
trained_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 43, 43, 64)        1792      
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 41, 41, 64)        36928     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 20, 20, 64)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 20, 20, 32)        18464     
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 18, 18, 32)        9248      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 9, 9, 32)          0         
__________